# Process Documentation
This notebook is meant to document past progress and attempts at exploring and visualizing the NFID4 Culture Knowledge Graph.

## Initial attempts
This section covers the first weeks of the project course and is meant to illustrate approaches and thought behind the work.
## Imagining a process pipeline
Initially the idea was to automate the querying of a [SPARQL](https://www.w3.org/TR/sparql11-query/) endpoint (in this case the [NFID4 Culture](https://nfdi4culture.de/resources/knowledge-graph.html) endpoint) and extracting the desired information and relations from that query to then display it using a visualization tool like [MELODY](https://melody-data.github.io/stories/).

This was not productive in the sense that the MELODY tool at the time was not working with the desired endpoint and that the idea was too analysis focused without regard for the relational properties of a knowledge graph as compared to a conventional database.

Consequently, the process pipeline has to be reimagined in a more recursive fashion and iterate over the graph s.t. it can meaningfully leverage the nature of a knowledge graph and enable exploration in a more natural sense. This process is inspired by approaches like [these](https://ome.dehio.org/de/start). % ADD MORE

A big part of this ideation process was familiarizing with the SPARQL basics and grasping the possibilities and limitations within the scope of the project.

## SPARQL Queries
Starting from the initially provided queries (\<insert here>) the process did at first not become productive as the queries did not work or were meaningless. A selection of the attempted queries can be found in `files/queries.json`.

The **working** SPARQL queries can now be found in the `files/sparql_queries` directory. Below is an example of a query that retrieves decade, composer and number of works of that composer in that decade.

```sql
    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX schema: <http://schema.org/>

    SELECT 
        ?decade 
        ?composer 
        ?composerName 
        (COUNT(?composition) AS ?numberOfCompositions)
    WHERE {
        ?composition rdf:type schema:MusicComposition .
        ?composition schema:dateCreated ?dateCreated .
        ?composition schema:composer ?composer .

        # Attempt to get human-readable name
        OPTIONAL { ?composer schema:name ?composerName . }
        
        # Fallback to rdfs:label if available
        OPTIONAL { ?composer rdfs:label ?composerName . }

        # Compute the decade
        BIND(FLOOR(YEAR(xsd:dateTime(?dateCreated)) / 10) * 10 AS ?decade)
    }
    GROUP BY 
        ?decade 
        ?composer 
        ?composerName
    ORDER BY 
        ?decade DESC 
        (?numberOfCompositions)
```